# 2.2 主流模型提供商对比

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/02-ai-landscape/model_providers.ipynb)

**预计 API 费用：~$0.01**

本 Notebook 演示如何调用 OpenAI GPT-4o-mini、Google Gemini 2.0 Flash 和 DeepSeek Chat，对比它们的回答质量、速度和 token 使用。

## 安装依赖

In [ ]:
!pip install -q openai google-genai

## 配置 API Keys

需要准备三个 API Key：
- **OpenAI API Key**：从 [OpenAI Platform](https://platform.openai.com/api-keys) 获取
- **Google API Key**：从 [Google AI Studio](https://aistudio.google.com/apikey) 获取
- **DeepSeek API Key**：从 [DeepSeek Platform](https://platform.deepseek.com/api-keys) 获取

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("请输入 OpenAI API Key: ")

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass("请输入 Google API Key: ")

if not os.environ.get("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass("请输入 DeepSeek API Key: ")

## 对比三个模型的回答

我们用同一个问题测试三个模型，对比它们的回答质量、速度和 token 用量。

In [ ]:
import time
from openai import OpenAI
from google import genai

# 测试问题
question = "用一个生活中的类比，解释什么是 API？要求：简洁、有趣、易懂"

print("=" * 60)
print("问题：", question)
print("=" * 60)

### 1. OpenAI GPT-4o-mini

In [ ]:
print("\n【OpenAI GPT-4o-mini】")

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

start_time = time.time()
response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": question}],
)
latency = time.time() - start_time

print(response.choices[0].message.content)
print(f"\n⏱️  延迟：{latency:.2f}s")
print(f"💰 Token 用量：输入 {response.usage.prompt_tokens}, 输出 {response.usage.completion_tokens}, 总计 {response.usage.total_tokens}")
print(f"💵 估算成本：${(response.usage.prompt_tokens * 0.15 + response.usage.completion_tokens * 0.60) / 1_000_000:.6f}")

### 2. Google Gemini 2.0 Flash

In [ ]:
print("\n【Google Gemini 2.0 Flash】")

google_client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

start_time = time.time()
response = google_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=question,
)
latency = time.time() - start_time

print(response.text)
print(f"\n⏱️  延迟：{latency:.2f}s")
print(f"💰 Token 用量：输入 {response.usage_metadata.prompt_token_count}, 输出 {response.usage_metadata.candidates_token_count}, 总计 {response.usage_metadata.total_token_count}")
print(f"💵 估算成本：${(response.usage_metadata.prompt_token_count * 0.075 + response.usage_metadata.candidates_token_count * 0.30) / 1_000_000:.6f}")

### 3. DeepSeek Chat

In [ ]:
print("\n【DeepSeek Chat】")

# DeepSeek 兼容 OpenAI API
deepseek_client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com",
)

start_time = time.time()
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[{"role": "user", "content": question}],
)
latency = time.time() - start_time

print(response.choices[0].message.content)
print(f"\n⏱️  延迟：{latency:.2f}s")
print(f"💰 Token 用量：输入 {response.usage.prompt_tokens}, 输出 {response.usage.completion_tokens}, 总计 {response.usage.total_tokens}")
print(f"💵 估算成本：${(response.usage.prompt_tokens * 0.27 + response.usage.completion_tokens * 1.10) / 1_000_000:.6f}")

## 进阶：批量对比多个模型

创建一个通用的对比函数，方便测试不同问题。

In [ ]:
def compare_models(question, test_cases):
    """
    对比多个模型的回答
    
    test_cases 格式：[
        {
            "name": "模型名称",
            "call": lambda: 调用模型的函数,
            "extract": lambda response: 提取回答和 token 的函数,
        },
    ]
    """
    print("=" * 80)
    print(f"问题：{question}")
    print("=" * 80)
    
    results = []
    
    for case in test_cases:
        print(f"\n【{case['name']}】")
        
        try:
            start_time = time.time()
            response = case["call"]()
            latency = time.time() - start_time
            
            answer, tokens = case["extract"](response)
            
            print(answer)
            print(f"\n⏱️  延迟：{latency:.2f}s | 💰 Tokens: {tokens}")
            
            results.append({
                "model": case["name"],
                "latency": latency,
                "tokens": tokens,
                "answer_length": len(answer),
            })
        
        except Exception as e:
            print(f"❌ 调用失败：{e}")
    
    # 打印对比总结
    if results:
        print("\n" + "=" * 80)
        print("对比总结：")
        print("=" * 80)
        for r in results:
            print(f"{r['model']:20s} | 延迟: {r['latency']:.2f}s | Tokens: {r['tokens']:4d} | 回答长度: {r['answer_length']} 字")
    
    return results

In [ ]:
# 定义测试用例
test_question = "用一句话解释什么是 Docker？"

test_cases = [
    {
        "name": "OpenAI GPT-4o-mini",
        "call": lambda: openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": test_question}],
        ),
        "extract": lambda r: (r.choices[0].message.content, r.usage.total_tokens),
    },
    {
        "name": "Google Gemini 2.0 Flash",
        "call": lambda: google_client.models.generate_content(
            model="gemini-2.0-flash",
            contents=test_question,
        ),
        "extract": lambda r: (r.text, r.usage_metadata.total_token_count),
    },
    {
        "name": "DeepSeek Chat",
        "call": lambda: deepseek_client.chat.completions.create(
            model="deepseek-chat",
            messages=[{"role": "user", "content": test_question}],
        ),
        "extract": lambda r: (r.choices[0].message.content, r.usage.total_tokens),
    },
]

# 运行对比
results = compare_models(test_question, test_cases)

## 动手练习

1. 修改 `question`，测试三个模型在不同任务上的表现（如写代码、写文案、数学推理）
2. 观察哪个模型速度最快？哪个回答最详细？哪个最便宜？
3. 尝试添加更多模型（如 Qwen、Claude）到对比函数中

## 小结

- **OpenAI GPT-4o-mini**：质量稳定，生态最好，价格中等
- **Google Gemini 2.0 Flash**：速度快，价格最低，免费额度大
- **DeepSeek Chat**：性价比高，推理能力强，中文友好

**选型建议：**
- 高质量场景：OpenAI GPT-4o / Claude Opus
- 性价比场景：Gemini Flash / DeepSeek / GPT-4o-mini
- 多模态场景：Gemini 2.0 Flash / GPT-4o

下一步：学习如何用聚合平台统一调用这些模型 → [2.3 聚合平台统一调用](./aggregators.ipynb)